In [ ]:
# ! pip install transformers==4.38.1
# ! pip install rdkit==2023.9.4
# ! pip install accelerate==0.27.2
# ! pip install tensorflow==2.10.0
# ! pip install flash-attn
# ! pip install -q -U bitsandbytes
# ! pip install datasets
# ! pip install loralib
# ! pip install git+https://github.com/huggingface/peft.git

In [1]:
import random
import pickle
import json
from datasets import Dataset
import torch
import torch.nn as nn
import bitsandbytes as bnb
from peft import PeftModelForCausalLM

import sys
sys.path.append('../credentials/')
from HF_credentials import *